### Introduccion a
# Solr

![Solr Logo](https://home.apache.org/~hossman/ac2014na/slide-resources/solr-logo.svg)


## Introducción a [Solr](http://lucene.apache.org/solr/)

- Motor/Plataforma de búsqueda  - incluye indexado en tiempo (casi) real
- Parte del Proyecto Apache 
- Open source (Licencia Apache 2)
- Basado en Apache Lucene - biblioteca de búsqueda de texto
- API HTTP - JSON/XML

## Solr
- Primera version pública - 2006 
- Version a 22 Febrero de 2015 - Solr 5.5.0
- Cuenta con: 
    - proyecto Apache - es uno de los proyectos estrella 
    - comunidad de desarrolladores - consolidada y numerosos commiters de Lucene
    - relaciones con otros proyectos Apache: Lucene, Nutch, Hadoop, UIMA, Tika
    - consultoras externas: e.g. Sematext 
    - elegido por distribuciones como Cloudera - Cloudera Search es 
- Recientemente han mejorado: 
   - Web 
   - Documentacion 
   - [Solr Github mirror](https://github.com/apache/lucene-solr/)

## Características

- Motor/Plataforma de Búsqueda (Enterprise Search Server)
- Indexacion en tiempo (casi) real
- Optimizado para gran volumen de búsquedas
- **Extensiones para indizar formatos documentales comunes - PDF, Word, Excel, etc**
- **Integrado con pipelines de procesamiento de lenguaje**
- UI Admin integrada
- **Búsqueda facetada**
- **UI Búsqueda integrada**

## SolrCloud
- Nace como un proyecto separado para montar clusters Solr 
- Ahora esta integrado dentro de Solr
- Motor de búsqueda distribuido
- Escalable a cientos de servidores y petabytes de datos, 
- Tolerante a fallos y particiones de red (**Zookeeper**) 


## Conceptos básicos Solr (vs ES) 

 - **Documentos semiestructurados** - igual que en ES, aunque nativamente tiene más formatos XML,JSON,CSV
 - **Campos** - aprox. mismos tipos
 - **Cores** -  conjunto de documentos del mismo tipo
      - similares a los índices pero su tipo es único
      - una instancia Solr puede tener varios cores
  - **Colecciones (Collections)** - cores en SolrCloud
       - pueden estar distribuidas (*shards*) y replicadas
      

## Conceptos básicos Solr (vs ES) 

- **Schema** - cada core requiere un esquema del documento vs *schemaless*
    - Ahora Solr tambien tiene la opcion de definir un core *schemaless* 
    - en los esquemas se pueden definir tipos y usar *convention over configuratio*
    - no hay creación de índices dinámicos 

- **Configuracion** 
    - toda la configuracion es por fichero vs API (ES) 
    - suele requerir rearrancar la instancia
 
 - API de consultas HTTP + parametros vs API REST
 

## Instalación
  - Bajar la distribucion - tb. requiere Java 
  - Arrancar Solr

<pre>
$ bin/solr start            # this starts solr
$ bin/solr create -c demo   # this creates a document collection called "demo"
</pre>
  
  - Disponible en el puerto 8983 con interfaz de admin: 
     - [http://localhost:8983/solr/](http://localhost:8983/solr/)


## Lenguaje de Consulta

 - Principalmente basado en parametros URL 
 - Ha incorporado un DSL en JSON/XML 

## Indexación

In [51]:
import requests


employee = """
[
{
    "id": 1,
    "first_name_t" : "John",
    "last_name_t" :  "Smith",
    "age_i" :        25,
    "about_t" :      "I love to go rock climbing",
    "interests_txt": [ "sports", "music" ]
}
]
"""

r = requests.post('http://localhost:8983/solr/megacorp/update/json', 
                 data = employee)
print r.text

{"responseHeader":{"status":0,"QTime":3}}



In [52]:
employee2 = """
[
{
    "id": 2,
    "first_name_t" :  "Jane",
    "last_name_t" :   "Smith",
    "age_i" :         32,
    "about_t" :       "I like to collect rock albums",
    "interests_txt":  [ "music" ]
},
{
    "id": 3,
    "first_name_t" :  "Douglas",
    "last_name_t" :   "Fir",
    "age_i" :         35,
    "about_t":        "I like to build cabinets",
    "interests_txt":  [ "forestry" ]
}
]
"""

r = requests.post('http://localhost:8983/solr/megacorp/update/json?commitWithin=1000', data = employee2)
print r.text

{"responseHeader":{"status":0,"QTime":2}}



###  Obteniendo el documento

In [53]:
r = requests.get("http://localhost:8983/solr/megacorp/get?id=1")
print r.text

{
  "doc":
  {
    "id":"1",
    "first_name_t":["John"],
    "last_name_t":["Smith"],
    "age_i":25,
    "about_t":["I love to go rock climbing"],
    "interests_txt":["sports",
      "music"],
    "_version_":1527005010689261568}}



### Consultando

In [54]:
r = requests.get("http://localhost:8983/solr/megacorp/query?q=last_name_t:Smith")
print r.text

{
  "responseHeader":{
    "status":0,
    "QTime":0,
    "params":{
      "q":"last_name_t:Smith"}},
  "response":{"numFound":2,"start":0,"docs":[
      {
        "id":"1",
        "first_name_t":["John"],
        "last_name_t":["Smith"],
        "age_i":25,
        "about_t":["I love to go rock climbing"],
        "interests_txt":["sports",
          "music"],
        "_version_":1527005010689261568},
      {
        "id":"2",
        "first_name_t":["Jane"],
        "last_name_t":["Smith"],
        "age_i":32,
        "about_t":["I like to collect rock albums"],
        "interests_txt":["music"],
        "_version_":1527005013354741760}]
  }}



### Consultando

In [58]:
r = requests.get("http://localhost:8983/solr/megacorp/query?q=about_t:rock climbing")
print r.text

{
  "responseHeader":{
    "status":0,
    "QTime":0,
    "params":{
      "q":"about_t:rock climbing"}},
  "response":{"numFound":2,"start":0,"docs":[
      {
        "id":"1",
        "first_name_t":["John"],
        "last_name_t":["Smith"],
        "age_i":25,
        "about_t":["I love to go rock climbing"],
        "interests_txt":["sports",
          "music"],
        "_version_":1527005010689261568},
      {
        "id":"2",
        "first_name_t":["Jane"],
        "last_name_t":["Smith"],
        "age_i":32,
        "about_t":["I like to collect rock albums"],
        "interests_txt":["music"],
        "_version_":1527005013354741760}]
  }}



In [60]:
r = requests.get("http://localhost:8983/solr/megacorp/query?q=about_t:rock climbing&fl=id,about_t,score")
print r.text

{
  "responseHeader":{
    "status":0,
    "QTime":0,
    "params":{
      "fl":"id,about_t,score",
      "q":"about_t:rock climbing"}},
  "response":{"numFound":2,"start":0,"maxScore":0.5036961,"docs":[
      {
        "id":"1",
        "about_t":["I love to go rock climbing"],
        "score":0.5036961},
      {
        "id":"2",
        "about_t":["I like to collect rock albums"],
        "score":0.108701006}]
  }}



## Configuracion

## Características distintivas
 - Comunidad de usuarios + basado en Lucene 
    - gran parte de las características de ES acaban estando disponibles en Solr 5.x
       - Nested Objects
       - Schemaless 
       - Facetas avanzadas - distinct counts
       - Tipos geográficos 
       - Query DSL en JSON 
 - Data Import Handler - indizado desde base de datos con una consulta 
 - Update request - Transformaciones en el documento previo a indexar 
 - Integración con formatos documentales (Apache Tika) : DOC, PDF, XML, XLS, PPT  
 - Integracion con herramientas de procesamiento de lenguaje: UIMA, GATE 
 - Velocity - interfaz sobre el buscador

## Usuarios de Solr
  - DuckDuckGo
  - Zappos
  - Sears
  - BestBuy
  - Bloomberg
  - ...
  

## Bibliografía

  - [Solr Tutorial](http://yonik.com/solr-tutorial/)
  - [Apache Solr Enterprise Search Server](https://www.packtpub.com/big-data-and-business-intelligence/apache-solr-enterprise-search-server-third-edition#), Third Edition by David Smiley, Eric Pugh, Kranti Parisa, and Matt Mitchell (Packt Publishing) May 2015.
  
  - Bastante bibliografía pero que cubre Solr v3 y v4. [Solr books - website](http://lucene.apache.org/solr/resources.html#solr-books)

### Referencia
 - [Official Solr Reference](https://www.apache.org/dyn/closer.lua/lucene/solr/ref-guide/)
